In [1]:
# !pip install lightgbm
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score

import bytedtqs
from pytqs import tqs

mpl.rcParams['figure.dpi'] = 300
sns.set_style("whitegrid")
pd.options.display.max_columns = None
%config Completer.use_jedi = False

## Data Cleaning

In [2]:
appId = "eL38NWjpAMXlC0ifCoo71wk27zE6CS9YQismEcwVH5oNKbJf"
appKey = "VeeOKtXLxjGWE2OyIGkuqWRT33isyxtHH1xlxakeFzId6SMB"
user_id = 'wuweiwei.1012'

sql_txt = """ 

set tqs.query.result.max.lines = 1000000000;

select * from dm_eo_test.new_fms_vip_v2 where user_segment not in ('Silent') limit 1000000000
"""

client = tqs.TQSClient(appId=appId, appKey=appKey)
job = client.execute_query(username = user_id , sql = sql_txt)
data = pd.read_csv(job['resultUrl'])

df_raw = data

[2021-11-05 10:27:31,867] - [INFO] - tqs_task_id: 475527049, engine: , status: Created, progress : Submitting
[2021-11-05 10:27:33,892] - [INFO] - tqs_task_id: 475527049, execute id: tqs_475527049-1636079252415
[2021-11-05 10:27:33,893] - [INFO] - application tracking url: http://presto-history.byted.org/proxy/20211105_022732_16525_rjhrf_default
[2021-11-05 10:27:33,893] - [INFO] - tqs_task_id: 475527049, engine: Presto, status: Processing, progress : Accepted
[2021-11-05 10:27:35,917] - [INFO] - tqs_task_id: 475527049, engine: Presto, status: Processing, progress : Accepted
[2021-11-05 10:27:37,948] - [INFO] - tqs_task_id: 475527049, engine: Presto, status: Processing, progress : Accepted
[2021-11-05 10:27:39,971] - [INFO] - tqs_task_id: 475527049, engine: Presto, status: Processing, progress : Accepted
[2021-11-05 10:27:41,991] - [INFO] - tqs_task_id: 475527049, engine: Presto, status: Processing, progress : Accepted
[2021-11-05 10:27:44,014] - [INFO] - tqs_task_id: 475527049, engine

[2021-11-05 10:31:28,860] - [INFO] - tqs_task_id: 475527049, engine: SparkCli, status: Processing, progress : jobId-0: 100%,jobId-1: 0%
[2021-11-05 10:31:33,890] - [INFO] - tqs_task_id: 475527049, engine: SparkCli, status: Processing, progress : jobId-0: 100%,jobId-1: 0%
[2021-11-05 10:31:38,910] - [INFO] - tqs_task_id: 475527049, engine: SparkCli, status: Processing, progress : jobId-0: 100%,jobId-1: 0%
[2021-11-05 10:31:43,929] - [INFO] - tqs_task_id: 475527049, engine: SparkCli, status: Processing, progress : jobId-0: 100%,jobId-1: 0%
[2021-11-05 10:31:49,147] - [INFO] - tqs_task_id: 475527049, engine: SparkCli, status: Processing, progress : jobId-0: 100%,jobId-1: 3%,jobId-2: 0%
[2021-11-05 10:31:54,174] - [INFO] - tqs_task_id: 475527049, engine: SparkCli, status: Processing, progress : jobId-0: 100%,jobId-1: 3%,jobId-2: 0%
[2021-11-05 10:31:59,452] - [INFO] - tqs_task_id: 475527049, engine: SparkCli, status: Processing, progress : jobId-0: 100%,jobId-1: 3%,jobId-2: 0%
[2021-11-05 

In [3]:
df_raw.head()

,user_id,study_dt,first_milestone_name,occupation,user_segment,user_vip_type,vip_lfc_video,vip_lfc_audio,vip_lfc_system,gambling_count,team_count,scholarship_status_video,scholarship_status_audio,scholarship_succeeded_audio,scholarship_succeeded_video,wemeet_count_all,finish_dubbing_lesson_cnt,words_review_count_all,study_duration_pre,aim_module_user,study_duration_next
0,6749348949,2021-10-25,未定级,unknown,Slient,非会员,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0.0
1,11803430241,2021-10-25,未定级,unknown,Slient,非会员,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0.0
2,14753293895,2021-10-25,未定级,unknown,Slient,非会员,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0.0
3,15020985888,2021-10-25,未定级,unknown,Slient,非会员,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0.0
4,15167617536,2021-10-25,未定级,unknown,Slient,非会员,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0.0


In [4]:
# fill na
category_cols = []

for c in df_raw.columns:
    if df_raw[c].dtype=='O':
        df_raw[c] = df_raw[c].fillna('unknown')
        category_cols.append(c)
    else:
        df_raw[c] = df_raw[c].fillna(0)

In [5]:
df_raw.groupby(['aim_module_user']).size()

aim_module_user
0    21299359
1       21965
dtype: int64

In [6]:
# add cum status
# df_sorted = df_raw\
#     .sort_values(by=['user_id', 'gap_days'])
# df_sorted['cum_var'] = np.where(df_sorted\
#     .groupby(['user_id'])['aim_module_user'].cumsum() > 0, 1, 0)
df_sorted = df_raw

In [11]:
# select training data
# _gap_days = 3
_x = [
    'occupation',
    'first_milestone_name',
    'user_vip_type',
    'vip_lfc_video',
    'vip_lfc_audio',
    'vip_lfc_system',
    'study_duration_pre',
    'aim_module_user',
    'gambling_count',
    'team_count',
    'scholarship_status_video',
    'scholarship_status_audio',
    'scholarship_succeeded_audio',
    'scholarship_succeeded_video',
    'wemeet_count_all',
    'finish_dubbing_lesson_cnt',
    'words_review_count_all',
]
categorial_x = [
    'first_milestone_name',
    'occupation',
    'user_vip_type',
]
_y = 'study_duration_next'

# df_train = df_sorted[lambda x: (x['enter_group_yn']==1)].reset_index()
df_train = df_sorted.reset_index()

## Standardizing

In [14]:
macth_summary = pd.DataFrame(columns = ["subset", "auc", "test", "control", "uplift", "test_st", "control_st"])

for _subset in ('HF', 'MF', 'LF'):
    df_train = df_sorted[lambda x: (x['user_segment']==_subset)].reset_index()
    
    # x & y
    train_x = df_train[_x]
    train_y = np.where(df_train[_y] > 0, 1, 0)

    # --------------- SEP ---------------
    # label encoding
    label_encoders = {}
    for c in categorial_x:
        encoder = LabelEncoder()
        train_x[c] = encoder.fit_transform(df_train[c].values)
        label_encoders[c] = encoder

    # --------------- SEP ---------------
    # counter-factual data
    train_x_cf_treated = train_x.assign(aim_module_user=lambda x: 1)
    train_x_cf_untreated = train_x.assign(aim_module_user=lambda x: 0)
    
    # training classifier
    mod = LGBMClassifier()
    mod.fit(train_x, train_y)
    
    y_pred_train = mod.predict(train_x)

    # predict counter-factual results
    pred_cf_treated = mod.predict_proba(train_x_cf_treated)[:,1]
    pred_cf_untreated = mod.predict_proba(train_x_cf_untreated)[:,1]
    
    # counter-factual mean diff
    mean_t = pred_cf_treated.mean()
    mean_u = pred_cf_untreated.mean()
    
    # actual mean diff
    df_stats = df_train.groupby(['aim_module_user']).apply(lambda x: pd.Series({
        'user_cnt': x.shape[0],
        'retention_rate': (x[_y]>0).mean()
    })).reset_index()
    
    macth_summary = macth_summary.append({"subset": _subset , \
                                          "auc": "{0:0.4f}". format(accuracy_score(train_y, y_pred_train)), \
                                          "test": "{:.2}".format(mean_t), \
                                          "control": "{:.2}".format(mean_u), \
                                          "uplift": "{:.2}".format(mean_t - mean_u),\
                                          "test_st": "{:.2}".format(df_stats.iloc[1,2]),\
                                          "control_st": "{:.2}".format(df_stats.iloc[0,2])\
                                         }, ignore_index = True)


macth_summary

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,subset,auc,test,control,uplift,test_st,control_st
0,HF,0.7740,0.56,0.64,-0.081,0.6,0.64
1,MF,0.9540,0.13,0.047,0.086,0.22,0.045
2,LF,0.9938,0.085,0.0061,0.079,0.13,0.0058


In [15]:
df_train = df_sorted.reset_index()

# x & y
train_x = df_train[_x]
train_y = np.where(df_train[_y] > 0, 1, 0)

# --------------- SEP ---------------
# label encoding
label_encoders = {}
for c in categorial_x:
    encoder = LabelEncoder()
    train_x[c] = encoder.fit_transform(df_train[c].values)
    label_encoders[c] = encoder

# --------------- SEP ---------------
# counter-factual data
train_x_cf_treated = train_x.assign(aim_module_user=lambda x: 1)
train_x_cf_untreated = train_x.assign(aim_module_user=lambda x: 0)

# training classifier
mod = LGBMClassifier()
mod.fit(train_x, train_y)

y_pred_train = mod.predict(train_x)

# predict counter-factual results
pred_cf_treated = mod.predict_proba(train_x_cf_treated)[:,1]
pred_cf_untreated = mod.predict_proba(train_x_cf_untreated)[:,1]

# counter-factual mean diff
mean_t = pred_cf_treated.mean()
mean_u = pred_cf_untreated.mean()

# actual mean diff
df_stats = df_train.groupby(['aim_module_user']).apply(lambda x: pd.Series({
    'user_cnt': x.shape[0],
    'retention_rate': (x[_y]>0).mean()
})).reset_index()

macth_summary = macth_summary.append({"subset": _subset , \
                                      "auc": "{0:0.4f}". format(accuracy_score(train_y, y_pred_train)), \
                                      "test": "{:.2}".format(mean_t), \
                                      "control": "{:.2}".format(mean_u), \
                                      "uplift": "{:.2}".format(mean_t - mean_u),\
                                      "test_st": "{:.2}".format(df_stats.iloc[1,2]),\
                                      "control_st": "{:.2}".format(df_stats.iloc[0,2])\
                                     }, ignore_index = True)


macth_summary

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,subset,auc,test,control,uplift,test_st,control_st
0,HF,0.7740,0.56,0.64,-0.081,0.6,0.64
1,MF,0.9540,0.13,0.047,0.086,0.22,0.045
2,LF,0.9938,0.085,0.0061,0.079,0.13,0.0058
3,LF,0.9977,0.021,0.004,0.017,0.47,0.0035


## IP Weighting

In [27]:
# x & y
train_x = df_train[_x]
train_y = df_train[_y]

# --------------- SEP ---------------
# one-hot encoding
encoder = OneHotEncoder(sparse=False)  

encoded_df = pd.DataFrame(
    encoder.fit_transform(train_x[categorial_x]), 
    columns=encoder.get_feature_names(categorial_x)
)

# combine onehot features
train_x = pd.concat([
        train_x.drop(columns=categorial_x),
        encoded_df
    ],
    axis=1,
)

def ip(y, X):
    mod = LogisticRegression(fit_intercept=True, C=5)
    mod.fit(X, y)
    weights = np.zeros(X.shape[0])
    weights[y == 1] = \
        mod.predict_proba(X.loc[y == 1])[:,1]
    weights[y == 0] = \
        mod.predict_proba(X.loc[y == 0])[:,0]
    return weights

# standardized weights
denoms = ip(
    train_x['cum_var'], 
    train_x.drop(columns=['cum_var'])
)

weights = 1 / denoms

treat_mask = (train_x['cum_var']==1)
s_weights = np.zeros(train_x.shape[0])
s_weights[treat_mask] = \
    treat_mask.mean() * weights[treat_mask]
s_weights[~treat_mask] = \
    (1 - treat_mask).mean() * weights[~treat_mask]

# fit gee model
train_x_a_1 = pd.DataFrame({
        'a':train_x['cum_var'], 
        'one':np.ones(train_x.shape[0])
    })
gee = sm.GEE(
    train_y,
    train_x_a_1,
    groups=train_x_a_1.index.tolist(),
    weights=s_weights
)

gee.fit().summary().tables[1]

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,coef,std err,z,P>|z|,[0.025,0.975]
a,-165.6434,148.026,-1.119,0.263,-455.770,124.483
one,171.5397,148.025,1.159,0.247,-118.585,461.664
